In [1]:
#17-MAR-2023 by Duane Rinehart (drinehart@ucsd.edu)
#PROCESS nifti FORMAT FILES (MEDICAL IMAGES)

#ref: https://nipy.org/nibabel/gettingstarted.html
#ref2: https://nipy.org/nibabel/images_and_memory.html
#ref3: https://github.com/nipy/nilabels


import nibabel as nib
import numpy as np


def sizeof_fmt(num, suffix="B"):
    '''
    src: https://stackoverflow.com/questions/1094841/get-human-readable-version-of-file-size
    '''
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"


def prepare_data(nib_image, calc_descriptive_stats):
    '''
    -DESCRIBES NIFI DATA ON DISK
    -CONVERTS TO NUMPY (BUT DOES NOT LOAD INTO MEMORY)
    -CAST NUMPY ARRAY TO int8
    -DESCRIBES NUMPY ARRAY
    '''
    hdr = nib_image.header
    storage_dtype = nib_image.get_data_dtype()

    #convert to numpy
    np_data = np.asarray(nib_image.dataobj).astype(np.int8)

    if calc_descriptive_stats == True:
        unique, frequency = np.unique(np_data, return_counts = True)
        rbytes = np_data.nbytes
        sparsity = 1 - (np.count_nonzero(raw_data) / raw_data.size)
        
        print(f'ORG SHAPE: {nib_image.shape}')
        print(f'ORG UNITS: {hdr.get_xyzt_units()}')    
        print(f'ORG DATA TYPE: {storage_dtype}')
        print('---')
        print(f'NUMPY MEM SIZE: {sizeof_fmt(rbytes)}')
        print(f'NUMPY dtype: {type(np_data.dtype)}')
        print('---')
        print(f'OVERALL - unique_values: {unique}, frequency: {frequency}')
        print(f'Sparsity of raw_data (zero/total):{np.round(sparsity,3)}%')

    return np_data


file = 'itk_WholeBrain_ML20190124_240_cube_downsampled_8x_0_0_0_mask_14mar2pm.nii.gz'
calc_descriptive_stats = False

nib_image = nib.load(file) #'proxy' image (not actually in RAM, loads from storage)
raw_data = prepare_data(nib_image, calc_descriptive_stats)


In [12]:
'''
NUMPY MEM SIZE: 4.5GiB
NUMPY dtype: <class 'numpy.dtype[uint16]'>
OVERALL - unique_values: [0 1 2 3], frequency: [2386845089   11267417      14481    1527413]

NUMPY MEM SIZE: 2.2GiB (2m34s to run)
NUMPY dtype: <class 'numpy.dtype[int8]'>

OTHER COMPRESSION RESULTS:
-> sparse array (COO)
NUMPY MEM SIZE: 305.4MiB
NUMPY dtype: <class 'numpy.dtype[int8]'>

-> sparse array (GCXS)
NUMPY MEM SIZE: 110.0MiB
NUMPY dtype: <class 'numpy.dtype[int8]'>

'''

"\nNUMPY MEM SIZE: 4.5GiB\nNUMPY dtype: <class 'numpy.dtype[uint16]'>\nOVERALL - unique_values: [0 1 2 3], frequency: [2386845089   11267417      14481    1527413]\n\nNUMPY MEM SIZE: 2.2GiB\nNUMPY dtype: <class 'numpy.dtype[int8]'>\n\n-> sparse array\nNUMPY MEM SIZE: 305.4MiB\nNUMPY dtype: <class 'numpy.dtype[int8]'>\n\n-> sparse array (6m10s to run)\nNUMPY MEM SIZE: 317.6MiB\nNUMPY dtype: <class 'numpy.dtype[uint16]'>\n"

In [7]:
#17-MAR-2023 CONVERT NUMPY ARRAY TO SPARSE ARRAY (COO)
import sparse

# sparsity = 1 - (np.count_nonzero(raw_data) / raw_data.size)
# print(f'Sparsity of raw_data:{np.round(sparsity,3)}%')

np_data_sparse = sparse.COO(raw_data)
print(f'SPARSE MEM SIZE: {sizeof_fmt(np_data_sparse.nbytes)}')

Sparsity of raw_data:0.995%
SPARSE MEM SIZE: 305.4MiB


In [2]:
#17-MAR-2023 CONVERT NUMPY ARRAY TO SPARSE ARRAY (GCXS)
#ref: https://ieeexplore.ieee.org/document/7237032
import sparse

sparsity = 1 - (np.count_nonzero(raw_data) / raw_data.size)
print(f'Sparsity of raw_data:{np.round(sparsity,3)}%')

np_data_sparse = sparse.GCXS(raw_data)
print(f'SPARSE MEM SIZE: {sizeof_fmt(np_data_sparse.nbytes)}')

Sparsity of raw_data:0.995%
SPARSE MEM SIZE: 110.0MiB


c:\ProgramData\Anaconda3\lib\site-packages\sparse\_compressed\compressed.py:76: RuntimeWarning: overflow encountered in long_scalars
  strides *= d


In [ ]:
sum=1040* 1320* 1748
val = 12809311
val/sum

0.005337981586015053